In [7]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms


In [12]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)



In [3]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_size = 28  
hidden_size = 128
num_layers = 2
num_classes = 10 

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [5]:
num_epochs = 5

for epoch in range(num_epochs):
    for batch_idx, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28, 28).to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (batch_idx+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item()}')


Epoch [1/5], Step [100/600], Loss: 1.057112216949463
Epoch [1/5], Step [200/600], Loss: 0.7718001008033752
Epoch [1/5], Step [300/600], Loss: 0.5272331833839417
Epoch [1/5], Step [400/600], Loss: 0.49489468336105347
Epoch [1/5], Step [500/600], Loss: 0.31297919154167175
Epoch [1/5], Step [600/600], Loss: 0.3762851059436798
Epoch [2/5], Step [100/600], Loss: 0.3604830801486969
Epoch [2/5], Step [200/600], Loss: 0.3134707510471344
Epoch [2/5], Step [300/600], Loss: 0.1543247103691101
Epoch [2/5], Step [400/600], Loss: 0.11392787843942642
Epoch [2/5], Step [500/600], Loss: 0.1703042834997177
Epoch [2/5], Step [600/600], Loss: 0.2715418338775635
Epoch [3/5], Step [100/600], Loss: 0.31445106863975525
Epoch [3/5], Step [200/600], Loss: 0.18109850585460663
Epoch [3/5], Step [300/600], Loss: 0.18144017457962036
Epoch [3/5], Step [400/600], Loss: 0.09752533584833145
Epoch [3/5], Step [500/600], Loss: 0.21019510924816132
Epoch [3/5], Step [600/600], Loss: 0.305891215801239
Epoch [4/5], Step [100